In [18]:
import numpy as np
import json
import os
import csv

In [19]:
#creating training & test data file: 18*2*45*n => 30 frame overlap

In [20]:
asanas = ["bhujangasan", "padamasan", "shavasan", "tadasan", "trikonasan", "vrikshasan"]
data_path = "D:\Kaustav_IITK\Others\Tushar\Yoga_data\pose_data"
# "D:\Kaustav_IITK\Others\Tushar\pose_data\bhujangasan"

In [21]:
train_list = []
#list of asanas, each asana has all it's videos, each video has list of stacked 18*2 numpy arrays
val_list = []
test_list = []
# print("ok")
for a in asanas:
    print("ok")
    currAsanaTrain = []
    currAsanaVal = []
    currAsanaTest = []
    path = data_path + "\\" +  a
    for i in range(1,17):
        # print("ok")
        currVideo = []
        start =  str(i) + "_"
        for filename in os.listdir(path):
            data = []
            # print(i)
            if filename.startswith(start):
                #get data from file
                # print("o")
                with open(path + "\\" + filename) as json_data:
                    # print("ok1")
                    d = json.load(json_data)
                    # print(d)
                    npdata = np.asarray(d)
                    # print(npdata.shape)
                    # print(npdata.shape)
                    Xdata = npdata[:,:,0]
                    Ydata = npdata[:,:,1]
                    # print(len(Ydata))
                    # print(Xdata)
                    stk = np.dstack((Xdata, Ydata)) #stack vertically
                    currVideo.append(stk)
                    # print(stk)
                # print("ok")
        #print currVideo
        if a == 'vrikshasan' and i == 15:
            # this one has difference and creates noise
            currAsanaTrain.append(currVideo)
        elif (i+1)%5 == 0 and len(currVideo) != 0:
            currAsanaTest.append(currVideo)
        elif (i)%5 == 0 and len(currVideo) != 0:
            currAsanaVal.append(currVideo)
        elif len(currVideo) != 0:
            currAsanaTrain.append(currVideo)
        # break
        # print("0")
    train_list.append(currAsanaTrain)
    val_list.append(currAsanaVal)
    test_list.append(currAsanaTest)
    # break

ok
ok
ok
ok
ok
ok


In [22]:
print(len(train_list))
print(len(train_list[0]))
print(len(train_list[0][0]))
print(len(train_list[0][0][0]))
print(len(train_list[0][0][0][0]))
# len(test_list[0])

6
10
1
54
33


In [23]:
# flatten_list = (train_list[0][0])
# nested_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
# flattened_list = [item for sublist in train_list for item in sublist]


In [24]:
# train_list[1] = [item for sublist in train_list[1] for item in sublist]
# train_list[0] = [item for sublist in train_list[0] for item in sublist]

In [25]:
#videos in datset are 30 fps
window_size = 10 # number of frames in 1 test case
# overlap_size = 36 # overlap between two testcase
train_cases = []
train_labels = []
sum=0
for i, asana in enumerate(train_list):
    for vid in asana:
        vid = vid[0]
        # window_size = ok
        print(len((vid)))
        sum+=len((vid))
        for start in range(0, len(vid) - window_size + 1):  # Adjusted the range to prevent IndexError
            currCase = np.empty([window_size,33,2])  # Updated shape to match vid
            for index in range(0, window_size):
                if(start+index >= len(vid)):
                    break # Added this line to prevent IndexError
                currCase[index] = vid[start+index]
            train_cases.append(currCase)
            train_labels.append(i)
            

54
140
121
146
126
125
177
81
92
147
86
133
133
128
131
125
76
140
132
130
159
123
126
160
129
196
90
92
151
155
122
126
128
125
153
91
162
132
151
126
140
137
124
124
122
147
177
155
153
146
134
126
156
154
160
125


In [26]:
if train_cases and train_labels:
    print(len(train_cases))
    print(len(train_labels))
else:
    print("train_cases or train_labels is empty")

6896
6896


In [27]:
test_cases = []
test_labels = []

for i, asana in enumerate(test_list):
    for vid in asana:
        vid = vid[0]
        for start in range(0, len(vid) - window_size + 1):
            currCase = np.empty([window_size,33,2])
            for index in range(0,window_size):
                if(start+index >= len(vid)):
                    break # Added this line to prevent IndexError
                currCase[index] = vid[start+index]
            test_cases.append(currCase)
            test_labels.append(i)

In [28]:
print (len(test_cases))
print (len(test_labels))

2030
2030


In [29]:
val_cases = []
val_labels = []
for i, asana in enumerate(val_list):
    for vid in asana:
        vid = vid[0]
        for start in range(0, len(vid) - window_size + 1):
            currCase = np.empty([window_size,33,2])
            for index in range(0,window_size):
                if(start+index >= len(vid)):
                    break # Added this line to prevent IndexError
                currCase[index] = vid[start+index]
            val_cases.append(currCase)
            val_labels.append(i)

In [30]:
print (len(val_cases))
print (len(val_labels))

2092
2092


In [31]:
#convert to numpy array and save
train_arr = np.empty([len(train_cases), window_size, 33, 2])
for i, ele in enumerate(train_cases):
    train_arr[i] = ele

In [32]:
test_arr = np.empty([len(test_cases), window_size, 33, 2])
for i, ele in enumerate(test_cases):
    test_arr[i] = ele

In [33]:
val_arr = np.empty([len(val_cases), window_size, 33, 2])
for i, ele in enumerate(val_cases):
    val_arr[i] = ele

In [34]:
np.save("trainX", train_arr)
np.save("testX", test_arr)
np.save("valX", val_arr)
np.save("trainY", np.asarray(train_labels))
np.save("testY", np.asarray(test_labels))
np.save("valY", np.asarray(val_labels))